<a href="https://www.kaggle.com/code/mafiosoquasar/sales-forecasting-with-pytorch?scriptVersionId=209641694" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [274]:
import argparse
import os
import random,numpy,pandas
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [275]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.transforms.functional as RF
from PIL import Image
import numpy as np
import random,cv2,pandas,os,numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [276]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

nz = 100
beta1 = 0.5
lr = 0.01
batch_size=100
ngpu=1
ngf,nc = 3,3
ndf = 64

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Random Seed:  999


In [ ]:
x = pandas.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv").sort_values(by=['date', 'warehouse'])

x = torch.from_numpy(numpy.nan_to_num(x[['total_orders', 'sell_price_main', 'type_0_discount', 'type_1_discount',
                                         'type_2_discount', 'type_3_discount', 'type_4_discount',  'type_5_discount', 
                                         'type_6_discount']].to_numpy().astype(numpy.float32)).reshape((13, 308263, 9)))
x.shape #4315682

class EC_NET(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rafire = nn.Sequential(       
            nn.Linear(9, 1524),
            nn.Linear(1524, 824),
            nn.Linear(824, 424),
            nn.Linear(424, 124),
            nn.Linear(124, 100)
        )

    def forward(self,x):
        
        return self.rafire(x)
ec_net = EC_NET().type(torch.float32).to(device).eval()
ec_net= nn.DataParallel(ec_net).to(device)
rf_net.load_state_dict(torch.load("/kaggle/input/encoder-weight-data/weight.pth",weights_only=False,map_location=torch.device('cpu')))

j=0
for i in x:
    print(f"loding batch : {j}")
    if j==0:
        encode = ec_net(i).cpu().detach().numpy().reshape((len(i),10,10))
        print(encode.shape)
    else:
        encode = numpy.concatenate((encode, ec_net(i).cpu().detach().numpy().reshape((len(i),10,10))), axis=0)
    #if j==25:
    #    break
    j+=1

x=torch.Tensor(encode)[::100]

In [277]:
y=pandas.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv").sort_values(by=['date', 'warehouse'])
y=torch.from_numpy(numpy.nan_to_num(y['sales'].to_numpy().astype(numpy.float32))).reshape(len(y),1)[::100]

/tmp/ipykernel_30/566033409.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x = torch.load("/kaggle/input/encoder-weight-data/train_tensor.pt")[::100]


In [278]:
x.shape, y.shape, x

(torch.Size([40075, 1]),
 torch.Size([40075, 1]),
 tensor([[25.9468],
         [25.3594],
         [26.0216],
         ...,
         [27.3667],
         [29.2135],
         [30.7281]]))

In [279]:
train_x = torch.utils.data.DataLoader(x,batch_size=batch_size)
train_y = torch.utils.data.DataLoader(y,batch_size=batch_size)

In [280]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=0.02)
        
class RF_NET(nn.Module):
    def __init__(self):
        super().__init__()
        self.rafire_1 = nn.LSTMCell(1, 1524)
                                     #nn.BatchNorm1d(1524),
                                     #nn.ReLU())

        self.rafire_2 = nn.LSTMCell(1524, 1324)
                                     #nn.BatchNorm1d(1324),
                                     #nn.LeakyReLU())

        self.rafire_3 = nn.LSTMCell(1324, 1024)
                                     #nn.BatchNorm1d(1024),
                                     #nn.LeakyReLU())

        self.rafire_4 = nn.LSTMCell(1024, 824)
                                     #nn.BatchNorm1d(824),
                                     #nn.LeakyReLU())

        self.rafire_5 = nn.LSTMCell(824, 524)
                                     #nn.BatchNorm1d(524),
                                     #nn.LeakyReLU())

        self.rafire_6 = nn.LSTMCell(524, 324)
                                     #nn.BatchNorm1d(324),
                                     #nn.LeakyReLU())

        self.rafire_7 = nn.LSTMCell(324, 124)
                                    #nn.BatchNorm1d(124),
                                    #nn.LeakyReLU())
            
        self.rafire_8 = nn.Linear(124, 1)

    def forward(self,x):
        x,_=self.rafire_1(x)
        x,_=self.rafire_2(x)
        x,_=self.rafire_3(x)
        x,_=self.rafire_4(x)
        x,_=self.rafire_5(x)
        x,_=self.rafire_6(x)
        x,_=self.rafire_7(x)
        
        return self.rafire_8(x)

In [281]:
rf_net = RF_NET().type(torch.float32).to(device)
rf_net= nn.DataParallel(rf_net).to(device)
if (device.type == 'cuda') and (ngpu > 1):
    rf_net = nn.DataParallel(rf_net, list(range(ngpu)))
rf_net.apply(weights_init)

DataParallel(
  (module): RF_NET(
    (rafire_1): LSTMCell(1, 1524)
    (rafire_2): LSTMCell(1524, 1324)
    (rafire_3): LSTMCell(1324, 1024)
    (rafire_4): LSTMCell(1024, 824)
    (rafire_5): LSTMCell(824, 524)
    (rafire_6): LSTMCell(524, 324)
    (rafire_7): LSTMCell(324, 124)
    (rafire_8): Linear(in_features=124, out_features=1, bias=True)
  )
)

In [282]:
criterion = nn.L1Loss()

optimizer = optim.AdamW(rf_net.parameters(), lr=lr,betas=(beta1, 0.999))
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.86)

In [283]:
high_rf,i,j_r,k,z_w_=100000,0,0,0,[]
if os.path.exists(f"/kaggle/working/jsr_rf")==False:
    os.mkdir(f"/kaggle/working/jsr_rf")
    
correct,total=0,0

while(True):
    optimizer.zero_grad()
    for data,label in zip(train_x,train_y):
        #z_ = nn.Embedding(10, 3, dtype=torch.float64)
        #print(z_(torch.FloatTensor(data)))
        
        output = rf_net(data.to(device)).view(-1)
        err_real = criterion(output, label.to(device))
        err_real.backward()
        optimizer.step()
        #print(err_real.item())
        
    print(f"EPOCH : {i} LOSS_wl : {err_real.item()}")

    if len(z_w_)>=3:
        if len([True for i in range(1,4) if z_w_[len(z_w_)-i]<=z_w_[len(z_w_)-3] and z_w_[len(z_w_)-i]>=z_w_[len(z_w_)-4]])==3:
            z_w_,j_r=[],0
            print(f"lr_br:{optimizer.param_groups[0]['lr']}".upper())
            scheduler.step()
            print(f"lr_up:{optimizer.param_groups[0]['lr']}".upper())
            
    z_w_.append(err_real.item())
    if err_real.item()<high_rf:
            high_rf=err_real.item()
            torch.save(rf_net.state_dict(),f'/kaggle/working/jsr_rf/{err_real.item()}.pth')
    if i==100:
        break
    i+=1

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([100, 1])) that is different to the input size (torch.Size([100])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([75, 1])) that is different to the input size (torch.Size([75])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


EPOCH : 0 LOSS_wl : 111.59806060791016
EPOCH : 1 LOSS_wl : 117.55054473876953
EPOCH : 2 LOSS_wl : 110.01910400390625
EPOCH : 3 LOSS_wl : 110.89949035644531
EPOCH : 4 LOSS_wl : 111.52262115478516
EPOCH : 5 LOSS_wl : 110.16775512695312
EPOCH : 6 LOSS_wl : 111.755615234375
EPOCH : 7 LOSS_wl : 110.42478942871094
EPOCH : 8 LOSS_wl : 111.56241607666016
EPOCH : 9 LOSS_wl : 110.26602172851562
LR_BR:0.01
LR_UP:0.0086
EPOCH : 10 LOSS_wl : 108.74249267578125
EPOCH : 11 LOSS_wl : 107.75145721435547
EPOCH : 12 LOSS_wl : 109.53330993652344
LR_BR:0.0086
LR_UP:0.007396
EPOCH : 13 LOSS_wl : 111.6804428100586
EPOCH : 14 LOSS_wl : 107.55406951904297
EPOCH : 15 LOSS_wl : 108.58694458007812
EPOCH : 16 LOSS_wl : 111.00308990478516
EPOCH : 17 LOSS_wl : 107.83633422851562
EPOCH : 18 LOSS_wl : 108.67230224609375
EPOCH : 19 LOSS_wl : 109.56366729736328
EPOCH : 20 LOSS_wl : 108.15509033203125
EPOCH : 21 LOSS_wl : 108.86256408691406
EPOCH : 22 LOSS_wl : 108.7469482421875
EPOCH : 23 LOSS_wl : 108.5175552368164
EPO